In [ ]:
import os
import datetime as dt
import pandas as pd
import time

# Files

**List all files in directory**

In [ ]:
path = os.getcwd()
files = os.listdir(r'C:\Users\KohMansf\Documents\STAMS_FILES\DB\POR\POR2020')
os.chdir(r'C:\Users\KohMansf\Documents\STAMS_FILES\DB\POR\POR2020')

In [ ]:
files_csv = [f for f in files if f[4:8] == '2020']

In [ ]:
df = pd.DataFrame()

for f in files_csv:
    data = pd.read_csv(f, parse_dates=['Attribute'],dayfirst=True)
    df = df.append(data)

df.head()

In [ ]:
# path = os.getcwd()
# files = os.listdir(
#     r'C:\Users\KohMansf\Documents\STAMS_FILES\DB\POR\POR2018')
# os.chdir(r'C:\Users\KohMansf\Documents\STAMS_FILES\DB\POR\POR2018')

# files_xlsx = [f for f in files if f[4:8] == '2018']

# df = pd.DataFrame()

# for f in files_xlsx:
#     data = pd.read_excel(f)
#     df = df.append(data)

# df.head()

# POR Cleaning

In [ ]:
df['Time'] = pd.to_datetime(df['Attribute'])

df['YYYYWW'] = df['Time'].apply(
    lambda x: str(x.isocalendar()[0]) + str(x.isocalendar()[1]).zfill(2))
df.head()

**Get region base on target location base on original POR excel file**

In [ ]:
dfreg = pd.read_excel(
    r'C:\Users\KohMansf\Documents\STAMS_FILES\DB\Country of Target Location\Country of Target Location.xlsx',
    sheet_name='Country of Target Location',
    na_filter=False,
    usecols='A:B')

In [ ]:
dfmerge = pd.merge(df, dfreg, on='Target Location')

dfmerge.head()

In [ ]:
dfmerge.drop(columns=['Attribute'])
dfmerge['Time'] = (dfmerge['Time'] - dt.datetime(1899, 12, 30)).dt.days
# dfmerge = dfmerge.drop(columns=['Attribute'])

dfmerge['Planning Part Group'] = dfmerge['Planning Part Group'].str.rstrip(
    'GROUP')

In [ ]:
mpa_dict = {
    'NKG-THAILAND': 'NKG Thailand',
    'NKG Thailand': 'NKG Thailand',
    'NKG-YUEYANG': 'NKG Yue Yang',
    'DSG-VIETNAM': 'DSG Vietnam',
    'DSG Vietnam': 'DSG Vietnam',
    'FLEX-PTP': 'Flex PTP Malasya',
    'Flex PTP Malasya': 'Flex PTP Malasya',
    'FLEX-ZHUHAI': 'Flex Zhuhai',
    'Flex Zhuhai': 'Flex Zhuhai',
    'FOXCONN': 'Foxconn ChongQing',
    'Foxconn ChongQing': 'Foxconn ChongQing',
    'Jabil Circuit De Chihuahua': 'Jabil Circuit De Chihuahua',
    'Jabil Circuit Netherlands BV': 'Jabil Circuit Netherlands BV'
}

In [ ]:
dfmerge['MPA'] = dfmerge['MPA'].map(lambda x: mpa_dict.get(x, x))
dfmerge.head()

# Output

In [ ]:
dffinal = dfmerge[[
    'Current Cycle on Display', 'Product Line', 'Platform',
    'Planning Part Group', 'Target Location', 'Planning Part', 'MPA', 'region',
    'Value', 'YYYYWW', 'Attribute'
]]
dffinal

In [ ]:
por_df = dffinal.rename(
    columns={
        "Current Cycle on Display": "Planning_Wk",
        "Product Line": "Product_Line",
        "Planning Part Group": "Program",
        "Target Location": "Target_Location",
        "Planning Part": "SKU",
        "Value": "Qty",
        "region": "Region",
        "Attribute": "DATES"
    })
por_df['QtyType'] = 'POR'
por_df.head()

In [ ]:
por_df.to_csv('2020_to_upload.csv', index=False)

In [ ]:
# por_df.shape

# por_df.isnull().any().any()